In [1]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime

In [2]:
os.listdir("inside_airbnb")

['neighbourhoods.geo.json',
 'reviews.csv',
 '.DS_Store',
 'neighbourhoods.geojson',
 'listings_summ.csv',
 'neighbourhoods.csv',
 'stations.geojson',
 'listings.csv',
 'calendar.csv',
 'reviews_summ.csv',
 'neighbourhoods.json']

In [3]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [4]:
for i in reviews.columns:
    print i

listing_id
id
date
reviewer_id
reviewer_name
comments


In [61]:
stopWords = set(stopwords.words('english'))

In [5]:
reviews['comment_length'] = reviews['comments'].str.len()

In [6]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,11204286,https://www.airbnb.com/rooms/11204286,20160706203047,2016-07-07,Family friendly/California king,"Aquatica Waterpark, Sleep train Amphitheater, ...","Walking to Aquatica Waterpark, Sleep train Amp...","Aquatica Waterpark, Sleep train Amphitheater, ...",none,NaN,...,10.0,f,NaN,"SAN DIEGO, SAN DIEGO TOURISM MARKETING DISTRIC...",f,moderate,f,f,2,4.57
1,7972006,https://www.airbnb.com/rooms/7972006,20160706203047,2016-07-07,Welcome to Sunset Suite,Your spacious room awaiting is with a Queen Si...,NaN,Your spacious room awaiting is with a Queen Si...,none,Getting around is easy. Very close to Eastlake...,...,10.0,f,NaN,NaN,f,strict,f,f,1,0.76
2,7727710,https://www.airbnb.com/rooms/7727710,20160706203047,2016-07-07,San Diego/Eastlake. Gated community,"This is an immaculate 3 bedroom, 2 1/2 bath co...",NaN,"This is an immaculate 3 bedroom, 2 1/2 bath co...",none,NaN,...,8.0,f,NaN,NaN,f,flexible,f,f,1,0.09
3,13124681,https://www.airbnb.com/rooms/13124681,20160706203047,2016-07-07,Townhome in Eastlake,This 2 Story TownHome is close to Otay Ranch ...,"My place is good for couples, business travele...",This 2 Story TownHome is close to Otay Ranch ...,none,"Located in eastern Chula Vista, Otay Ranch is ...",...,10.0,f,NaN,NaN,f,moderate,f,f,1,3.00
4,3469225,https://www.airbnb.com/rooms/3469225,20160706203047,2016-07-07,Bedroom suite in Large new home,Hello; we are offering a private secluded bedr...,"Beautiful, quiet award-winning suburban neighb...",Hello; we are offering a private secluded bedr...,none,"The quiet serenity; near Park and lakes, beaut...",...,NaN,f,NaN,NaN,f,strict,f,f,2,NaN
5,877473,https://www.airbnb.com/rooms/877473,20160706203047,2016-07-07,Private Bedroom in Newer Home,"Private Bedroom with full size bed, + futon, p...","Private bedroom with full size Bed, in large n...","Private Bedroom with full size bed, + futon, p...",none,"It is quiet, serene, tranquil, suburban; yet c...",...,10.0,f,NaN,NaN,f,strict,f,f,2,0.63
6,3124507,https://www.airbnb.com/rooms/3124507,20160706203047,2016-07-07,"Great view to mountains,lake,trails","In this beautiful home, located near the Olymp...",The location and view of lake and mountain,"In this beautiful home, located near the Olymp...",none,Like countryside and also near from downtown S...,...,10.0,f,NaN,NaN,f,strict,f,f,3,0.31
7,3432507,https://www.airbnb.com/rooms/3432507,20160706203047,2016-07-07,2 bedrooms with shared bath,In a great house with view on Otay Lakes and ...,"the proximity to lake ,trail and Olympic train...",In a great house with view on Otay Lakes and ...,none,Freedom in nature,...,10.0,f,NaN,NaN,f,strict,f,f,3,0.04
8,3249729,https://www.airbnb.com/rooms/3249729,20160706203047,2016-07-07,Near to Olympic Training center,Beautiful home in the Eastlake community with ...,Location on the border of the town with nice t...,Beautiful home in the Eastlake community with ...,none,"open , quiet, country feeling",...,10.0,f,NaN,NaN,f,strict,f,f,3,0.08
9,11756336,https://www.airbnb.com/rooms/11756336,20160706203047,2016-07-07,Chula Vista Golf Course House,"Feel right at home in our spacious 5BR, 3.5BA ...","Feel right at home in our spacious 5BR, 3.5BA ...","Feel right at home in our spacious 5BR, 3.5BA ...",none,Our home is surrounded by natural beauty. Visi...,...,8.0,f,NaN,NaN,f,strict,f,f,5,0.59


In [7]:
listings.columns

Index([u'id', u'listing_url', u'scrape_id', u'last_scraped', u'name',
       u'summary', u'space', u'description', u'experiences_offered',
       u'neighborhood_overview', u'notes', u'transit', u'access',
       u'interaction', u'house_rules', u'thumbnail_url', u'medium_url',
       u'picture_url', u'xl_picture_url', u'host_id', u'host_url',
       u'host_name', u'host_since', u'host_location', u'host_about',
       u'host_response_time', u'host_response_rate', u'host_acceptance_rate',
       u'host_is_superhost', u'host_thumbnail_url', u'host_picture_url',
       u'host_neighbourhood', u'host_listings_count',
       u'host_total_listings_count', u'host_verifications',
       u'host_has_profile_pic', u'host_identity_verified', u'street',
       u'neighbourhood', u'neighbourhood_cleansed',
       u'neighbourhood_group_cleansed', u'city', u'state', u'zipcode',
       u'market', u'smart_location', u'country_code', u'country', u'latitude',
       u'longitude', u'is_location_exact', u'prope

In [8]:
#Confirmation that all ids are unique 
listings['id'].nunique()

6608

In [9]:
#Using 11204286 as an example. Number of reviews is 7.
listings[listings['id'] == 11204286]['number_of_reviews'] 

0    7
Name: number_of_reviews, dtype: int64

In [10]:
#taking the number of reviews in the reviews file for id 11204286 (check if they are equal)
len(reviews[reviews['listing_id'] == 11204286])

7

In [11]:
sum(reviews[reviews['listing_id'] == 11204286]['comment_length'])

1954.0

In [12]:
#Checking if there are no NaN in the review count (the length is the same as the original length)
len(listings['number_of_reviews'].dropna())

6608

In [13]:
#Average review length is sum(length_reviews)/count(reviews)
listings['average_review_length'] = 0

for idx in listings['id'].index:
    w = listings['id'][idx]
    a = listings[listings['id'] == w]['number_of_reviews'].item()
    
    if a == 0:
        continue
        
    b = sum(reviews[reviews['listing_id'] == w]['comment_length'])
    listings['average_review_length'][idx] = float(b)/float(a)
    

C:\Users\sanka\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sanka\Anaconda2\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
#Computing a weight based on reviewer's history
L = []
for w in reviews['reviewer_id'].unique():
    a = len(reviews[reviews['reviewer_id'] == w])
    if a > 1:
        L.append((w, a))
    
L

[(61280862, 3),
 (3628691, 3),
 (38839815, 2),
 (2804750, 2),
 (68072119, 3),
 (15746017, 2),
 (57166342, 2),
 (18004048, 2),
 (56405589, 2),
 (9913594, 2),
 (43625266, 2),
 (33480788, 2),
 (1009467, 2),
 (23700601, 2),
 (46179252, 6),
 (27714399, 2),
 (13430797, 2),
 (7275138, 2),
 (8354170, 3),
 (11101380, 2),
 (14368568, 2),
 (8479607, 2),
 (17602325, 2),
 (9650481, 2),
 (44670009, 2),
 (25012727, 3),
 (19532074, 2),
 (66281042, 3),
 (35734676, 2),
 (10175086, 4),
 (65940377, 2),
 (50155059, 2),
 (44751786, 2),
 (9037735, 2),
 (13223821, 2),
 (18383086, 2),
 (3377035, 6),
 (25774740, 3),
 (41525019, 2),
 (59357545, 2),
 (71495097, 2),
 (75206075, 2),
 (2824843, 6),
 (51691026, 3),
 (67221522, 2),
 (18093312, 3),
 (26919407, 2),
 (324652, 3),
 (26127554, 2),
 (27748750, 3),
 (2592550, 2),
 (5634438, 3),
 (2733694, 2),
 (11291312, 2),
 (27564373, 3),
 (283119, 2),
 (46364, 3),
 (46077279, 2),
 (14410864, 7),
 (59453156, 2),
 (46183205, 9),
 (45361366, 2),
 (13387816, 5),
 (31159455, 2

In [15]:
#Converting list to dictionary 
reviewerid_count_dict = {}

for w in L:
    reviewerid_count_dict[w[0]] = w[1]

In [16]:
#Creating the reviewer_weight column 
reviews['reviewer_weight'] = 1

for w in reviews.index:
    a = reviews['reviewer_id'][w].item()
    
    if a in reviewerid_count_dict.keys():
        b = reviewerid_count_dict[a]
        reviews['reviewer_weight'][w] = b

C:\Users\sanka\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
#Checking if it went well (and it did)
reviews['reviewer_weight'].unique()

array([ 1,  3,  2,  6,  4,  7,  9,  5, 17, 10, 15, 19,  8, 27, 11, 14, 12,
       26], dtype=int64)

In [18]:
def compute_comment_stopword_ratio(x):
    count = 0
    x = str(x)
    x_list = x.strip(',').split(' ')
    
    for word in x_list:
        if word in stopWords:
            count = count + 1
            
    return float(count)/len(x_list)        

In [171]:
reviews['comments_stop_words_ratio'] = reviews['comments'].map(compute_comment_stopword_ratio)

In [20]:
reviews[reviews['reviewer_id'] == 8222312]['listing_id'].unique()

array([2464739, 1919622, 2142779,  519919], dtype=int64)

In [21]:
reviewer_listing_pairs = reviews[['listing_id', 'reviewer_id', 'reviewer_name']]

In [22]:
reviewer_listing_pairs.head(10)

,listing_id,reviewer_id,reviewer_name
0,4796400,29916246,Bill
1,4796400,3547281,Christina
2,4796400,16458913,Naif
3,4796400,3199045,Matt
4,4796400,47834158,Sally
5,6716557,67892303,Mattie
6,6716557,58474306,Gretchen
7,12493871,57841638,Stefan
8,12493871,61280862,Taryn
9,12493871,72024589,SaraJo


In [23]:
reviewer_listing_pairs.groupby(['listing_id']).count().head(10)

,reviewer_id,reviewer_name
listing_id,,
6,44,44
9553,2,2
14668,165,165
37149,3,3
38245,79,79
39516,241,241
45429,6,6
54001,87,87
62274,299,299


In [24]:
len(reviewer_listing_pairs.groupby(['listing_id']).count())

4590

In [25]:
reviewer_listing_pairs.groupby(['reviewer_id']).count().head(10)

,listing_id,reviewer_name
reviewer_id,,
248,1,1
415,1,1
899,1,1
1467,1,1
1767,1,1
2288,1,1
2363,1,1
2425,1,1
2675,1,1


In [26]:
len(reviewer_listing_pairs.groupby(['reviewer_id']).count())

85263

In [27]:
reviewer_listing_pairs.groupby(['listing_id', 'reviewer_id']).count().head()

reviewer_name
listing_id reviewer_id               
6          415                      1
           3762                     1
           52739                    1
           117966                   1
           127886                   1

In [28]:
len(reviewer_listing_pairs.groupby(['listing_id', 'reviewer_id']).count())

91718

In [29]:
len(reviews)

92862

In [30]:
reviews['date'].min(), reviews['date'].max()

('2008-06-22', '2016-07-06')

In [31]:
reviews['id'].min(), reviews['id'].max()

(8, 84333214)

In [107]:
a = pd.datetime.now()

In [33]:
a

datetime.datetime(2018, 2, 22, 11, 18, 30, 254000)

In [34]:
a.month

2

In [48]:
def review_age(date):
    n = pd.datetime.now()
    
    if date.month > n.month:
        return (n.year - date.year - 1)
    
    return n.year - date.year

In [191]:
reviews['review_age'] = pd.to_datetime(reviews['date']).map(lambda x: review_age(x))

In [192]:
reviews['day_of_year'] = reviews['date'].map(lambda x: datetime.datetime.strptime(x,'%Y-%M-%d').timetuple().tm_yday)

In [145]:
reviews['day_of_year'].head(10)

0    20
1    25
2     7
3    28
4     2
5    16
6    31
7    11
8    17
9    24
Name: day_of_year, dtype: int64

In [62]:
datetime.datetime.now().timetuple().tm_yday

53

In [63]:
def day_of_year_elapsed(a):
    
    if a < datetime.datetime.now().timetuple().tm_yday:
        return (a - datetime.datetime.now().timetuple().tm_yday + 365.25)
    
    return (a - datetime.datetime.now().timetuple().tm_yday)

In [64]:
reviews['days_of_year_elapsed'] = reviews['day_of_year'].map(lambda x: day_of_year_elapsed(x))

In [65]:
reviews['review_age'] = reviews['review_age'] + (reviews['days_of_year_elapsed']/365.25)

In [43]:
reviews['list_reviwer_pair_count'] = 0 

for idx in reviews.index:
    list_id = reviews['listing_id'][idx].item()
    reviewer_id = reviews['reviewer_id'][idx].item()
    a = len(reviews[(reviews['listing_id'] == list_id) & (reviews['reviewer_id'] == reviewer_id)])
    reviews['list_reviwer_pair_count'][idx] = a

C:\Users\sanka\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [199]:
review_pair_count_dataframe_dict = {}

for val in reviews['list_reviwer_pair_count'].unique():
    review_pair_count_dataframe_dict[val] = (reviews[reviews['list_reviwer_pair_count'] == val])

In [46]:
for w in review_pair_count_dataframe_dict.keys():
    print(w, len(review_pair_count_dataframe_dict[w]))

(1, 90822)
(2, 1536)
(3, 273)
(4, 68)
(5, 40)
(6, 12)
(7, 14)
(8, 16)
(9, 9)
(10, 10)
(12, 12)
(13, 13)
(17, 17)
(20, 20)


In [144]:
reviews.head(10)

,listing_id,id,date,reviewer_id,reviewer_name,comments,comment_length,reviewer_weight,comments_stop_words_ratio,review_age,day_of_year,days_of_year_elapsed,list_reviwer_pair_count
0,4796400,32687022,2015-05-20,29916246,Bill,"Jeff, we cannot thank you enough for your hosp...",657.0,1,0.396552,2.904175,20,330.25,1
1,4796400,33062004,2015-05-25,3547281,Christina,"The location was amazing, we were able to chea...",265.0,1,0.428571,2.917864,25,335.25,1
2,4796400,41311067,2015-08-07,16458913,Naif,I would like to thank Jeff for his great hospi...,220.0,1,0.395349,2.868583,7,317.25,1
3,4796400,57640415,2015-12-28,3199045,Matt,Jeff was an amazing host! We rented all three ...,722.0,1,0.398374,2.926078,28,338.25,1
4,4796400,58316432,2016-01-02,47834158,Sally,Jeff was the consummate host. He was quick to...,677.0,1,0.427350,2.854894,2,312.25,1
5,6716557,74703862,2016-05-16,67892303,Mattie,"The location was excellent, literally ste...",313.0,1,0.403226,1.893224,16,326.25,1
6,6716557,77373202,2016-05-31,58474306,Gretchen,Wonderful beach house. So close to the beach. ...,101.0,1,0.363636,1.934292,31,341.25,1
7,12493871,73952032,2016-05-11,57841638,Stefan,perfect place for me and my family. would love...,59.0,1,0.416667,1.879535,11,321.25,1
8,12493871,80329505,2016-06-17,61280862,Taryn,Ryan and his staff were amazing!!!!! So incred...,139.0,3,0.363636,1.895962,17,327.25,1
9,12493871,81695415,2016-06-24,72024589,SaraJo,My communication with Ryan before our stay was...,1685.0,1,0.505917,1.915127,24,334.25,1


In [67]:
datetime.datetime.strptime(reviews['date'][2],'%Y-%M-%d').timetuple()

time.struct_time(tm_year=2015, tm_mon=1, tm_mday=7, tm_hour=0, tm_min=8, tm_sec=0, tm_wday=2, tm_yday=7, tm_isdst=-1)

In [71]:
datetime.datetime.strptime(reviews['date'][2],'%Y-%M-%d')

datetime.datetime(2015, 1, 7, 0, 8)

In [73]:
reviews['date'][2]

'2015-08-07'

In [131]:
def amenities_string_to_set(x):
    a = x[1:-1].split(',')
    c = {b for b in a}
    return c

In [132]:
listings['amenities_set'] = listings['amenities'].map(amenities_string_to_set)

In [135]:
total_amenities = set()

for asset_set in listings['amenities_set']:
    total_amenities = total_amenities.union(asset_set)

In [136]:
total_amenities

{'',
 '"24-Hour Check-in"',
 '"Air Conditioning"',
 '"Buzzer/Wireless Intercom"',
 '"Cable TV"',
 '"Carbon Monoxide Detector"',
 '"Elevator in Building"',
 '"Family/Kid Friendly"',
 '"Fire Extinguisher"',
 '"First Aid Kit"',
 '"Free Parking on Premises"',
 '"Hair Dryer"',
 '"Hot Tub"',
 '"Indoor Fireplace"',
 '"Laptop Friendly Workspace"',
 '"Lock on Bedroom Door"',
 '"Other pet(s)"',
 '"Pets Allowed"',
 '"Pets live on this property"',
 '"Safety Card"',
 '"Smoke Detector"',
 '"Smoking Allowed"',
 '"Suitable for Events"',
 '"Wheelchair Accessible"',
 '"Wireless Internet"',
 '"translation missing: en.hosting_amenity_49"',
 '"translation missing: en.hosting_amenity_50"',
 'Breakfast',
 'Cat(s)',
 'Dog(s)',
 'Doorman',
 'Dryer',
 'Essentials',
 'Gym',
 'Hangers',
 'Heating',
 'Internet',
 'Iron',
 'Kitchen',
 'Pool',
 'Shampoo',
 'TV',
 'Washer'}

In [137]:
len(total_amenities)

43

In [138]:
total_amenities_dict = {}
count = 0

for c in total_amenities:
    total_amenities_dict[c] = count
    count = count + 1

In [140]:
def amenities_one_hot_encode(x):
    vec = np.zeros((len(total_amenities), 1))
    
    for asset in x:
        idx = total_amenities_dict[asset]
        vec[idx] = 1
        
    return vec    

In [146]:
listings['amenities_encoding'] = listings['amenities_set'].map(amenities_one_hot_encode)

In [148]:
listings['amenities_encoding'].head(10)

0    [[0.0], [0.0], [1.0], [1.0], [0.0], [0.0], [0....
1    [[0.0], [0.0], [1.0], [1.0], [0.0], [0.0], [1....
2    [[0.0], [0.0], [1.0], [1.0], [0.0], [0.0], [0....
3    [[0.0], [0.0], [1.0], [0.0], [0.0], [1.0], [0....
4    [[0.0], [0.0], [1.0], [0.0], [0.0], [1.0], [0....
5    [[0.0], [0.0], [1.0], [0.0], [0.0], [1.0], [0....
6    [[0.0], [0.0], [1.0], [1.0], [0.0], [0.0], [0....
7    [[0.0], [0.0], [0.0], [0.0], [0.0], [1.0], [0....
8    [[0.0], [0.0], [0.0], [0.0], [0.0], [1.0], [0....
9    [[0.0], [0.0], [1.0], [0.0], [0.0], [0.0], [0....
Name: amenities_encoding, dtype: object